# 4️⃣ Zero-Shot Cross-Lingual Transfer using Adapters

Beyond AdapterFusion, which we trained in [the previous notebook](https://github.com/Adapter-Hub/adapter-transformers/blob/master/notebooks/04_Cross_Lingual_Transfer.ipynb), we can compose adapters for zero-shot cross-lingual transfer between tasks. We will use the stacked adapter setup presented in **MAD-X** ([Pfeiffer et al., 2020](https://arxiv.org/pdf/2005.00052.pdf)) for this purpose.

In this example, the base model is a pre-trained multilingual **XLM-R** (`xlm-roberta-base`) ([Conneau et al., 2019](https://arxiv.org/pdf/1911.02116.pdf)) model. Additionally, two types of adapters, language adapters and task adapters, are used. Here's how the MAD-X process works in detail:

1. Train language adapters for the source and target language on a language modeling task. In this notebook, we won't train them ourselves but use [pre-trained language adapters from the Hub](https://adapterhub.ml/explore/text_lang/).
2. Train a task adapter on the target task dataset. This task adapter is **stacked** upon the previously trained language adapter. During this step, only the weights of the task adapter are updated.
3. Perform zero-shot cross-lingual transfer. In this last step, we simply replace the source language adapter with the target language adapter while keeping the stacked task adapter.

Now to our concrete example: we select **XCOPA** ([Ponti et al., 2020](https://ducdauge.github.io/files/xcopa.pdf)), a multilingual extension of the **COPA** commonsence reasoning dataset ([Roemmele et al., 2011](https://people.ict.usc.edu/~gordon/publications/AAAI-SPRING11A.PDF)) as our target task. The setup is trained on the original **English** dataset and then transferred to **Chinese**.

## Installation

Besides `adapter-transformers`, we use HuggingFace's `datasets` library for loading the data. So let's install both first:

In [17]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="6"
# os.environ["CUDA_VISIBLE_DEVICES"]="-1"

import torch
print(torch.cuda.device_count())

1


In [18]:
SEED = 42
MODEL_DIR = './training_output/deu_disrpt_vanilla_bert'
BERT_MODEL = 'bert-base-german-cased'
BATCH_SIZE = 8

## Dataset Preprocessing

We need the English COPA dataset for training our task adapter. It is part of the SuperGLUE benchmark and can be loaded via `datasets` using one line of code:

In [19]:
import pandas as pd
from datasets import Dataset

def read_df_custom(file):
    header = 'doc     unit1_toks      unit2_toks      unit1_txt       unit2_txt       s1_toks s2_toks unit1_sent      unit2_sent      dir     nuc_children    sat_children    genre   u1_discontinuous        u2_discontinuous       u1_issent        u2_issent       u1_length       u2_length       length_ratio    u1_speaker      u2_speaker      same_speaker    u1_func u1_pos  u1_depdir       u2_func u2_pos  u2_depdir       doclen  u1_position      u2_position     percent_distance        distance        lex_overlap_words       lex_overlap_length      unit1_case      unit2_case      label'
    extracted_columns = ['unit1_txt', 'unit1_sent', 'unit2_txt', 'unit2_sent', 'dir', 'label', 'distance', 'u1_depdir', 'u2_depdir', 'u2_func', 'u1_position', 'u2_position', 'sat_children', 'nuc_children', 'genre', 'unit1_case', 'unit2_case',
                            'u1_discontinuous', 'u2_discontinuous', 'same_speaker', 'lex_overlap_length', 'u1_func']
    header = header.split()
    df = pd.DataFrame(columns=extracted_columns)
    file = open(file, 'r')

    rows = []
    count = 0 
    for line in file:
        line = line[:-1].split('\t')
        count+=1
        if count ==1: continue
        row = {}
        for column in extracted_columns:
            index = header.index(column)
            row[column] = line[index]
        rows.append(row)

    df = pd.concat([df, pd.DataFrame.from_records(rows)])
    return df

en_train_dataset_df = Dataset.from_pandas(read_df_custom('../../processed/eng.rst.rstdt_train_enriched.rels'))
en_test_dataset_df = Dataset.from_pandas(read_df_custom('../../processed/eng.rst.rstdt_test_enriched.rels'))
en_valid_dataset_df = Dataset.from_pandas(read_df_custom('../../processed/eng.rst.rstdt_dev_enriched.rels'))

de_train_dataset_df = Dataset.from_pandas(read_df_custom('../../processed/deu.rst.pcc_train_enriched.rels'))
de_test_dataset_df = Dataset.from_pandas(read_df_custom('../../processed/deu.rst.pcc_test_enriched.rels'))
de_valid_dataset_df = Dataset.from_pandas(read_df_custom('../../processed/deu.rst.pcc_dev_enriched.rels'))

len(en_train_dataset_df), len(en_test_dataset_df), len(en_valid_dataset_df)

(16002, 2155, 1621)

In [20]:
en_train_dataset_df.features

{'unit1_txt': Value(dtype='string', id=None),
 'unit1_sent': Value(dtype='string', id=None),
 'unit2_txt': Value(dtype='string', id=None),
 'unit2_sent': Value(dtype='string', id=None),
 'dir': Value(dtype='string', id=None),
 'label': Value(dtype='string', id=None),
 'distance': Value(dtype='string', id=None),
 'u1_depdir': Value(dtype='string', id=None),
 'u2_depdir': Value(dtype='string', id=None),
 'u2_func': Value(dtype='string', id=None),
 'u1_position': Value(dtype='string', id=None),
 'u2_position': Value(dtype='string', id=None),
 'sat_children': Value(dtype='string', id=None),
 'nuc_children': Value(dtype='string', id=None),
 'genre': Value(dtype='string', id=None),
 'unit1_case': Value(dtype='string', id=None),
 'unit2_case': Value(dtype='string', id=None),
 'u1_discontinuous': Value(dtype='string', id=None),
 'u2_discontinuous': Value(dtype='string', id=None),
 'same_speaker': Value(dtype='string', id=None),
 'lex_overlap_length': Value(dtype='string', id=None),
 'u1_func':

In [21]:
# from datasets import load_dataset
from transformers.adapters.composition import Stack

# dataset_en = load_dataset("super_glue", "copa")
# dataset_en.num_rows

Every dataset sample has a premise, a question and two possible answer choices:

In [22]:
# dataset_en['train'].features

In this example, we model COPA as a multiple-choice task with two choices. Thus, we encode the premise and question as well as both choices as one input to our `xlm-roberta-base` model. Using `dataset.map()`, we can pass the full dataset through the tokenizer in batches:

In [23]:
# from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

# def encode_batch(examples):
#   """Encodes a batch of input data using the model tokenizer."""
#   all_encoded = {"input_ids": [], "attention_mask": []}
#   # Iterate through all examples in this batch
#   for premise, question, choice1, choice2 in zip(examples["premise"], examples["question"], examples["choice1"], examples["choice2"]):
#     sentences_a = [premise + " " + question for _ in range(2)]
#     # Both answer choices are passed in an array according to the format needed for the multiple-choice prediction head
#     sentences_b = [choice1, choice2]
#     encoded = tokenizer(
#         sentences_a,
#         sentences_b,
#         max_length=60,
#         truncation=True,
#         padding="max_length",
#     )
#     all_encoded["input_ids"].append(encoded["input_ids"])
#     all_encoded["attention_mask"].append(encoded["attention_mask"])
#   return all_encoded

# def preprocess_dataset(dataset):
#   # Encode the input data
#   dataset = dataset.map(encode_batch, batched=True)
#   # The transformers model expects the target class column to be named "labels"
#   # dataset.rename_column("label", "labels")
#   # Transform to pytorch tensors and only output the required columns
#   dataset.set_format(columns=["input_ids", "attention_mask", "label"])
#   return dataset

# dataset_en = preprocess_dataset(dataset_en)

In [24]:
from transformers import AutoTokenizer, BertTokenizer
from datasets import ClassLabel


# tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL)
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
class SNLIDataset(torch.utils.data.Dataset):
    """A customized dataset to load the SNLI dataset."""
    def __init__(self, dataset, labels, raw_text=False):
        self.text = []
        self.raw_text = []
        self.raw_label = []
        self.raw_text_flag = raw_text
        self.num_rows = len(dataset)
        for premise, hypothesis in zip(dataset['unit1_txt'], dataset['unit2_txt']):
            self.text.append(tokenizer.encode_plus(premise, hypothesis, padding="max_length", truncation=True, max_length=512, return_token_type_ids=True))
            if raw_text: self.raw_text.append([premise, hypothesis])
        # self.labels = torch.tensor(labels.str2int(dataset['label'])).to(device)
        self.labels = labels.str2int(dataset['label'])
        if raw_text: self.raw_label = dataset['label']
        print('read ' + str(len(self.text)) + ' examples')

    def __getitem__(self, idx):
        if self.raw_text_flag:  
            return {'input_ids':self.text[idx]['input_ids'], 
                'token_type_ids':self.text[idx]['token_type_ids'], 
                'attention_mask':self.text[idx]['attention_mask'], 
                'raw_text': self.raw_text[idx],
                'label':self.labels[idx],
                'raw_label': self.raw_label[idx]}

        return {'input_ids':self.text[idx]['input_ids'], 
                'token_type_ids':self.text[idx]['token_type_ids'], 
                'attention_mask':self.text[idx]['attention_mask'], 
                'label':self.labels[idx]}

    def __len__(self):
        return len(self.text)

    def combine_with_another_dataset(self, dataset2):
        print('Adding datasets of sizes ', self.num_rows, ', ', dataset2.num_rows)
        self.text = self.text + dataset2.text
        self.raw_text = self.raw_text + dataset2.raw_text
        self.raw_label = self.raw_label + dataset2.raw_label
        self.num_rows = len(self.text) #update texts is now longer
        assert self.raw_text_flag == dataset2.raw_text_flag
        print('To new dataset size: ', self.num_rows)


def load_data_snli(batch_size, train_dataset_df, valid_dataset_df, test_dataset_df, labels):
    """Download the SNLI dataset and return data iterators and vocabulary."""
    train_data = train_dataset_df
    valid_data = valid_dataset_df
    test_data = test_dataset_df
    train_set = SNLIDataset(train_data, labels, raw_text=False)
    valid_set = SNLIDataset(valid_data, labels, raw_text=False)
    test_set = SNLIDataset(test_data, labels, raw_text=False)
    return train_set, valid_set, test_set #TODO: MAKE INTO DICT

en_labels = ClassLabel(names=list(set(en_train_dataset_df['label'])|set(en_test_dataset_df['label'])|set(en_valid_dataset_df['label'])))
en_train_dataset, en_valid_dataset, en_test_dataset = load_data_snli(BATCH_SIZE, en_train_dataset_df, en_valid_dataset_df, en_test_dataset_df, en_labels)
de_labels = ClassLabel(names=list(set(de_train_dataset_df['label'])|set(de_test_dataset_df['label'])|set(de_valid_dataset_df['label'])))
de_train_dataset, de_valid_dataset, de_test_dataset = load_data_snli(BATCH_SIZE, de_train_dataset_df, de_valid_dataset_df, de_test_dataset_df, de_labels)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/xlm-roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/87683eb92ea383b0475fecf99970e950a03c9ff5e51648d6eee56fb754612465.dfaaaedc7c1c475302398f09706cbb21e23951b73c6e2b3162c1c8a99bb3b62a
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.2

read 16002 examples
read 1621 examples
read 2155 examples
read 2164 examples
read 241 examples
read 260 examples


## Task Adapter Training

In this section, we will train the task adapter on the English COPA dataset. We use a pre-trained XLM-R model from HuggingFace and instantiate our model using `AutoAdapterModel`.

In [25]:
from transformers import AutoConfig, AutoAdapterModel

config = AutoConfig.from_pretrained(
    "xlm-roberta-base",
)
model = AutoAdapterModel.from_pretrained(
    "xlm-roberta-base",
    config=config,
)

loading configuration file https://huggingface.co/xlm-roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/87683eb92ea383b0475fecf99970e950a03c9ff5e51648d6eee56fb754612465.dfaaaedc7c1c475302398f09706cbb21e23951b73c6e2b3162c1c8a99bb3b62a
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.21.3",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

loading weights 

This is the base AutoModel. If you print model before adding the adapters, you can see that the embeddings+12 transformer layers will be shared by En and Zh adapters.

Now we only need to set up the adapters. As described, we need two language adapters (which are assumed to be pre-trained in this example) and a task adapter (which will be trained in a few moments).

First, we load both the language adapters for our source language English (`"en"`) and our target language Chinese (`"zh"`) from the Hub. Then we add a new task adapter (`"copa"`) for our target task.

Finally, we add a multiple-choice head with the same name as our task adapter on top.

In [26]:
from transformers import AdapterConfig

# Load the language adapters
lang_adapter_config = AdapterConfig.load("pfeiffer", reduction_factor=2)
model.load_adapter("en/wiki@ukp", config=lang_adapter_config)
model.load_adapter("de/wiki@ukp", config=lang_adapter_config)

# Add a new task adapter
model.add_adapter("disrpt")

# Add a classification head for our target task
num_labels=de_labels.num_classes
print([num_labels])
# model.add_multiple_choice_head("disrpt", num_choices=num_labels)
model.add_classification_head("disrpt", num_labels=num_labels)

Attempting to load adapter from source 'ah'...
Found matching adapter at: adapters/ukp/xlm-roberta-base-en-wiki_pfeiffer.json
Resolved adapter files at https://public.ukp.informatik.tu-darmstadt.de/AdapterHub/text_lang/en/xlm-roberta-base/pfeiffer/en_relu_2.zip.
Loading module configuration from ~/.cache/torch/adapters/3e9c8c800af886cd1c684a3acf4929990b35f3ac75e050e942de2ff2ecf895fe-caabba7772816577124631d5b84810c4d32f49afcea4fca637162ec7f432174f-extracted/adapter_config.json
Adding adapter 'en'.
Loading module weights from ~/.cache/torch/adapters/3e9c8c800af886cd1c684a3acf4929990b35f3ac75e050e942de2ff2ecf895fe-caabba7772816577124631d5b84810c4d32f49afcea4fca637162ec7f432174f-extracted/pytorch_adapter.bin
No matching prediction head found in '~/.cache/torch/adapters/3e9c8c800af886cd1c684a3acf4929990b35f3ac75e050e942de2ff2ecf895fe-caabba7772816577124631d5b84810c4d32f49afcea4fca637162ec7f432174f-extracted'
Attempting to load adapter from source 'ah'...
Found matching adapter at: adapters/

[26]


Using `train_adapter()`, we tell our model to only train the task adapter in the following. This call will freeze the weights of the pre-trained model and the weights of the language adapters to prevent them from further finetuning.

In [27]:
model.train_adapter(["disrpt"])

We want the task adapter to be stacked on top of the language adapter, so we have to tell our model to use this setup via the `active_adapters` property.

A stack of adapters is represented by the `Stack` class, which takes the names of the adapters to be stacked as arguments.
Of course, there are various other possibilities to compose adapters beyonde stacking. Learn more about those [in our documentation](https://docs.adapterhub.ml/adapter_composition.html).

In [28]:
# Unfreeze and activate stack setup
model.active_adapters = Stack("en", "disrpt")

Great! Now, the input will be passed through the English language adapter first and the COPA task adapter second in every forward pass.

For training, we make use of the `Trainer` class built-in into `transformers`. We configure the training process using a `TrainingArguments` object.

As the dataset splits of English COPA in the SuperGLUE are slightly different, we train on both the train and validation split of the dataset. Later, we will evaluate on the test split of XCOPA.

In [29]:
# TODO: SHUFFLE print smples: https://discuss.huggingface.co/t/how-to-print-a-few-examples-at-the-beginning-of-training-when-using-trainer/7597/2
# https://github.com/huggingface/datasets/blob/cd3169f3f35afcf73a36a8276113e1881d92e5e0/src/datasets/iterable_dataset.py#L223

In [30]:
from transformers import TrainingArguments, AdapterTrainer
# from datasets import concatenate_datasets
from itertools import chain


def concatenate_datasets(dataset1, dataset2, shuffle=True):
    y_iter = chain(dataset1, dataset2)
    return y_iter
    # tmp = list(yielding(y_iter))
    # random.shuffle(tmp)
    # for i in tmp:
    #     print i

training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=8,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=100,
    output_dir=MODEL_DIR,
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

# train_dataset.combine_with_another_dataset(valid_dataset)
# train_dataset = concatenate_datasets([dataset_en["train"], dataset_en["validation"]])

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=en_train_dataset,
    eval_dataset=en_valid_dataset
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


# Train

Start the training 🚀 (this will take a while)

In [31]:
print(model)

XLMRobertaAdapterModel(
  (shared_parameters): ModuleDict()
  (roberta): RobertaModel(
    (shared_parameters): ModuleDict()
    (invertible_adapters): ModuleDict(
      (en): NICECouplingBlock(
        (F): Sequential(
          (0): Linear(in_features=384, out_features=192, bias=True)
          (1): Activation_Function_Class(
            (f): ReLU()
          )
          (2): Linear(in_features=192, out_features=384, bias=True)
        )
        (G): Sequential(
          (0): Linear(in_features=384, out_features=192, bias=True)
          (1): Activation_Function_Class(
            (f): ReLU()
          )
          (2): Linear(in_features=192, out_features=384, bias=True)
        )
      )
      (de): NICECouplingBlock(
        (F): Sequential(
          (0): Linear(in_features=384, out_features=192, bias=True)
          (1): Activation_Function_Class(
            (f): ReLU()
          )
          (2): Linear(in_features=192, out_features=384, bias=True)
        )
        (G): Sequen

In [32]:
trainer.train()

/home/VD/kaveri/anaconda3/envs/py310/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 16002
  Num Epochs = 8
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 4008
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss
100,2.297700
200,2.108000
300,2.052000
400,1.889900
500,1.818600
600,1.676000
700,1.609600
800,1.492300
900,1.432600
1000,1.366300


Saving model checkpoint to ./training_output/deu_disrpt_vanilla_bert/checkpoint-500
Configuration saved in ./training_output/deu_disrpt_vanilla_bert/checkpoint-500/en/adapter_config.json
Module weights saved in ./training_output/deu_disrpt_vanilla_bert/checkpoint-500/en/pytorch_adapter.bin
Configuration saved in ./training_output/deu_disrpt_vanilla_bert/checkpoint-500/de/adapter_config.json
Module weights saved in ./training_output/deu_disrpt_vanilla_bert/checkpoint-500/de/pytorch_adapter.bin
Configuration saved in ./training_output/deu_disrpt_vanilla_bert/checkpoint-500/disrpt/adapter_config.json
Module weights saved in ./training_output/deu_disrpt_vanilla_bert/checkpoint-500/disrpt/pytorch_adapter.bin
Configuration saved in ./training_output/deu_disrpt_vanilla_bert/checkpoint-500/disrpt/head_config.json
Module weights saved in ./training_output/deu_disrpt_vanilla_bert/checkpoint-500/disrpt/pytorch_model_head.bin
Configuration saved in ./training_output/deu_disrpt_vanilla_bert/checkpo

TrainOutput(global_step=4008, training_loss=1.29226510776969, metrics={'train_runtime': 3224.538, 'train_samples_per_second': 39.701, 'train_steps_per_second': 1.243, 'total_flos': 4.008443861031322e+16, 'train_loss': 1.29226510776969, 'epoch': 8.0})

In [33]:
import numpy as np
from transformers import EvalPrediction

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}
eval_trainer = AdapterTrainer(
    model=model,
    args=TrainingArguments(output_dir="./eval_output_en", remove_unused_columns=False,),
    eval_dataset=en_test_dataset,
    compute_metrics=compute_accuracy,
)
eval_trainer.evaluate()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running Evaluation *****
  Num examples = 2155
  Batch size = 8


Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


{'eval_loss': 1.2091971635818481,
 'eval_acc': 0.6477958236658933,
 'eval_runtime': 27.9135,
 'eval_samples_per_second': 77.203,
 'eval_steps_per_second': 9.673}

## Cross-lingual transfer

With the model and all adapters trained and ready, we can come to the cross-lingual transfer step here. We will evaluate our setup on the Chinese split of the XCOPA dataset.
Therefore, we'll first download the data and preprocess it using the same method as the English dataset:

In [34]:
# dataset_zh = load_dataset("xcopa", "zh", ignore_verifications=True)
# dataset_zh = preprocess_dataset(dataset_zh)
# print(dataset_zh["test"][0])

Next, let's adapt our setup to the new language. We simply replace the English language adapter with the Chinese language adapter we already loaded previously. The task adapter we just trained is kept. Again, we set this architecture using `active_adapters`:

In [35]:
model.active_adapters = Stack("de", "disrpt")

Finally, let's see how well our adapter setup performs on the new language. We measure the zero-shot accuracy on the test split of the target language dataset. Evaluation is also performed using the built-in `Trainer` class.

In [36]:
import numpy as np
from transformers import EvalPrediction

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}
eval_trainer = AdapterTrainer(
    model=model,
    args=TrainingArguments(output_dir="./eval_output", remove_unused_columns=False,),
    eval_dataset=de_test_dataset,
    compute_metrics=compute_accuracy,
)
eval_trainer.evaluate()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running Evaluation *****
  Num examples = 260
  Batch size = 8


Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


{'eval_loss': 6.6125311851501465,
 'eval_acc': 0.038461538461538464,
 'eval_runtime': 3.432,
 'eval_samples_per_second': 75.757,
 'eval_steps_per_second': 9.615}

In [37]:
import numpy as np
from transformers import EvalPrediction

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}
eval_trainer = AdapterTrainer(
    model=model,
    args=TrainingArguments(output_dir="./eval_output_en", remove_unused_columns=False,),
    eval_dataset=en_test_dataset,
    compute_metrics=compute_accuracy,
)
eval_trainer.evaluate()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running Evaluation *****
  Num examples = 2155
  Batch size = 8


Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


{'eval_loss': 1.2605915069580078,
 'eval_acc': 0.6264501160092807,
 'eval_runtime': 27.9991,
 'eval_samples_per_second': 76.967,
 'eval_steps_per_second': 9.643}

You should get an overall accuracy of about 56 which is on-par with full finetuning on COPA only but below the state-of-the-art which is sequentially finetuned on an additional dataset before finetuning on COPA.

For results on different languages and a sequential finetuning setup which yields better results, make sure to check out [the MAD-X paper](https://arxiv.org/pdf/2005.00052.pdf).

So you mean to say that even the SOTAs struggle to match randomized baseline for Zh.